In [1]:
import pandas as pd 
import cpi

In [2]:
df = pd.read_csv('../updated_clean_money_data_1980-2020.csv')

In [6]:
def inflate_money(data):
    pd.set_option('display.float_format', '{:.0f}'.format)
    normalized_lst = []
    for index, row in data.iterrows():
        if row['YEAR'] < 2020:
            normalized_lst += [cpi.inflate(row['money'], row['YEAR'])]
        else:
            normalized_lst += [row['money']]
        
    
    data['normalized'] = normalized_lst
    return data

In [7]:
current = inflate_money(df[['YEAR','name','money']])

/Users/connorlockman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [8]:
current

,YEAR,name,money,normalized
0,1980,Tom Watson,530808,1667220
1,1980,Lee Trevino,385814,1211807
2,1980,Curtis Strange,271888,853976
3,1980,Andy Bean,269033,845008
4,1980,Ben Crenshaw,237727,746679
5,1980,Jerry Pate,222976,700348
6,1980,George Burns,219928,690774
7,1980,Craig Stadler,206291,647942
8,1980,Mike Reid,206097,647332
9,1980,Raymond Floyd,192993,606174


In [9]:
# make records for all time
def all_time_earnings(data):
    year_lst = []
    for i in range(1980,2021):
        year_lst += [i]
        
    player_lst = data.groupby(['name']).size().reset_index()
    
    final_df = pd.DataFrame()
    new_name = []
    new_year = []
    for i in player_lst['name']:
        for y in year_lst:
            new_name += [i]
            new_year += [y]
    final_df['name'] = new_name
    final_df['YEAR'] = new_year

    # merge historical player list with all years to actual df with earnings 
    new = final_df.merge(data,how='left', on=['name','YEAR'])
    new = new.reset_index()
    new['normalized'] = new['normalized'].fillna(0)
    
    # update the historical data to include normalized cumulative sums of earnings 
    output = pd.DataFrame()
    for i in player_lst['name']:
        w_df = new.loc[new['name']==i]
        w_df = w_df.sort_values(by="YEAR")
        w_df['normalized_cumsum'] = w_df['normalized'].cumsum()    
        #print(w_df)
        output = output.append(w_df)
    
    return output

In [10]:
all_time_earnings_data = all_time_earnings(current)

In [11]:
all_time_earnings_data.to_csv('new_all_time_earnings_data.csv')